In [5]:
%pip install sentence-transformers qdrant-client[fastembed] polars folium transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 3.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 5.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 6.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 6.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 6.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 7.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 7.4 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 7.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing

In [6]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer
import polars as pl
import numpy as np
from tqdm import tqdm

In [7]:
qdrant_client = QdrantClient(":memory:")

In [8]:
qdrant_client.set_model("sentence-transformers/all-MiniLM-L6-v2")

TypeError: 'NoneType' object is not callable

In [18]:
qdrant_client.recreate_collection(
    collection_name="pois",
    vectors_config=qdrant_client.get_fastembed_vector_params(),
)

True

In [16]:
def convert(row):
    return [
        "category: " + row["category"],
        "sub category: " + row["sub_category"],
        "name: " + row["name"],
        "description: " + row["description"] if row["description"] else "",
        "brand: " + row["brand"] if row["brand"] else "",
        "alt_name: " + row["alt_name"] if row["alt_name"] else "",

    ]

In [17]:
df_geo = pl.read_parquet("../data/transformed/poi_clean_category_geo.parquet")
df = df_geo.drop(["type", "geometry"])
df.head()

id,lat,lon,addr:city,addr:country,addr:housenumber,addr:postcode,addr:street,brand,brand:wikidata,brand:wikipedia,cuisine,description,drive_through,name,note,operator,takeaway,wheelchair,bicycle,information,access,wikidata,wikimedia_commons,wikipedia,website,toilets:wheelchair,alt_name,check_date,opening_hours,changing_table,contact:phone,contact:website,delivery,diet:vegetarian,indoor_seating,internet_access,…,phone,heritage,heritage:operator,image,man_made,level,operator:wikidata,wifi,old_name,seamark:harbour:category,seamark:name,seamark:type,harbour,highway,name:fr,direction,capacity,reservation,smoking,toilets:access,backrest,colour,material,seats,delivery:covid19,opening_hours:covid19,takeaway:covid19,description:covid19,payment:cash,payment:maestro,payment:mastercard,contact:mobile,url,contact:email,addr:suburb,sub_category,category
i64,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
23656136,49.858471,6.3649698,null,null,null,null,null,null,null,null,null,null,null,"""Waldsportplatz…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""fitness_statio…","""leisure"""
26860223,49.743478,6.0898523,null,null,null,null,null,null,null,null,null,null,null,"""Restaurant Cam…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""restaurant""","""amenity"""
30432808,50.856197,5.8256972,null,null,null,null,null,null,null,null,null,null,null,"""Steenkolenmijn…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""attraction""","""tourism"""
31425173,50.844029,5.6890789,null,null,null,null,null,null,null,null,null,null,null,"""Nieuwenhofpoor…",null,null,null,null,null,null,null,"""Q19630871""","""Category:Nieuw…","""nl:Nieuwenhofp…",null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""city_gate""","""historic"""
31575884,50.844551,5.6902818,null,null,null,null,null,null,null,null,null,null,null,"""Universiteitsb…",null,null,null,null,null,null,null,"""Q15734302""",null,"""nl:Universitei…","""https://librar…",null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""library""","""amenity"""


In [21]:
qdrant_client.add(
    collection_name="pois",
    documents=[convert(row) for row in df.to_dicts()],
    metadata=[row for row in df.to_dicts()],
    ids=tqdm(range(df.shape[0])),
)

100%|██████████| 63330/63330 [29:38<00:00, 35.61it/s]  


[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [22]:
def add_loc(row):
    row["location"] = {"lat": row["lat"], "lon": row["lon"]}
    return row

In [23]:
class NeuralSearcher:
    def __init__(self, collection_name):
        self.collection_name = collection_name
        # Initialize encoder model
        self.model = SentenceTransformer("all-MiniLM-L6-v2", device="cuda")
        # initialize Qdrant client
        self.qdrant_client = qdrant_client
        self.qdrant_client.set_model("sentence-transformers/all-MiniLM-L6-v2")

    def search(self, pos, neg, around):
        # Use `vector` for search for closest vectors in the collection
        search_result = self.qdrant_client.recommend(
            collection_name=self.collection_name,
            positive=pos,
            negative=neg,
            strategy=models.RecommendStrategy.AVERAGE_VECTOR,
            query_filter=models.Filter(
                must=[
                    models.FieldCondition(
                        key="location",
                        geo_radius=models.GeoRadius(
                            center=models.GeoPoint(
                                lon=around[1],
                                lat=around[0],
                            ),
                            radius=1000.0,
                        ),
                    ),
                ],
                # must_not=[
                #     models.FieldCondition(
                #         key="category",
                #         match=models.MatchValue(
                #             value="amenity",
                #         ),
                #     )
                # ],
            ),
            limit=5,  # 5 the most closest results is enough
        )
        # `search_result` contains found vector ids with similarity scores along with the stored payload
        # In this function you are interested in payload only
        payloads = [hit.payload for hit in search_result]
        return payloads

In [24]:
neural_searcher = NeuralSearcher(collection_name="pois")

AttributeError: 'tokenizers.Tokenizer' object has no attribute 'get_added_tokens_decoder'

In [190]:
pos = [409005527, 488390305, 491571270, 490992842]
df.select(["id", "name", "category", "sub_category"]).filter(pl.col("id").is_in(pos))

id,name,category,sub_category
i64,str,str,str
409005527,"""Le Gros Vélo""","""amenity""","""cafe"""
488390305,"""La croix St-Cl…","""historic""","""wayside_cross"""
490992842,"""La Roche qui T…","""tourism""","""attraction"""
491571270,"""Château du Lav…","""historic""","""castle"""


In [191]:
neg = [494911197, 498293266, 494911196, 516617417]
df.select(["id", "name", "category", "sub_category"]).filter(pl.col("id").is_in(neg))

id,name,category,sub_category
i64,str,str,str
494911196,"""FrietJess""","""amenity""","""fast_food"""
494911197,"""Pizza Plaza""","""amenity""","""restaurant"""
498293266,"""Frit city juni…","""amenity""","""fast_food"""
516617417,"""Pizza Hut""","""amenity""","""restaurant"""


In [193]:
items = neural_searcher.search(
    pos=pos,
    neg=neg,
    around=[50.467388, 4.871985],
)
[
    [
        item["id"],
        item["name"],
        item["lat"],
        item["lon"],
        item["category"],
        item["sub_category"],
    ]
    for item in items
]
len(items)

5

In [185]:
import folium

# create a map centered on the item's location
m = folium.Map(location=[items[0]["lat"], items[0]["lon"]], zoom_start=15)

# add a marker for the item's location
for item in items:
    folium.Marker(
        location=[item["lat"], item["lon"]], popup=str(item["id"]) + " " + item["name"]
    ).add_to(m)

# display the map
m